In [ ]:
# Implement the Modeling class in Home-Credit-Prediction/homecredit/model.py

In [1]:

import os
import sys
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency # need this for chi-squared function
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path_dir = (os.path.dirname(os.getcwd()))
sys.path.append(path_dir)
    
from homecredit.data import HomeCredit
from homecredit.preparation import Preparation
from homecredit.cleaner import Cleaning
from homecredit.exploration import Exploration

In [4]:
#.remove_missvalues() method
# removes first some entries
# then, Replaces the NaNs in numerical column by the mean of values
# in numerical column respectively 
# for categorical variables, NaNs are replaced by ''
df = Cleaning().remove_missvalues()
df.shape

(91206, 122)

In [35]:
catcols = Cleaning().catcols # Preparation().get_catcols() # categorical columns
numcols = Cleaning().numcols # Preparation().get_numcols() # numerical columns

In [36]:
#data = df.drop_duplicates(subset = df.columns)
#data.shape
#There are no duplicates values

# Encoding function

In [8]:
# N.B : Here, we use OneHotEncoder, but we also can use 
#pandas.get_dummies. More, get_dummies easier than OneHotEncoder

def encoding_categ_column(df, cols):
    
    for col_name in cols:
    
        L = list(df[col_name].unique())
        if '' in L:
            df[col_name].replace("", "NoValue", inplace=True) #Replace NaN by "NoCodeNature"

        ohe = OneHotEncoder(sparse = False) # Instanciate encoder
        ohe.fit(df[[col_name]]) # Fit encoder  ---> OneHotEncoder(sparse=False)

        col_encoded = ohe.transform(df[[col_name]]) # Encode

        dicts_col = {}
        keys = list(ohe.categories_[0])
        values = col_encoded.T.astype(int)

        for i,j in enumerate(keys):
            dicts_col[j] = values[i,:]

        result = pd.DataFrame.from_dict(dicts_col)

        df = df.reset_index(drop=True)

        #Concat df and result dataframes
        data_res = pd.concat([df, result], axis = 1)

        if 'NoValue' in list(data_res.columns):
            data_res = data_res.drop(columns= ['NoValue',col_name] )
            df = data_res
        else:
            data_res = data_res.drop(columns= col_name)
            df = data_res
        
    return df

In [9]:
encoded_df = encoding_categ_column(df = df,
                                   cols = catcols)
#encoded_df.shape

# KNeighborsClassifier() model

In [14]:
# create X, y
y = encoded_df.TARGET
X = encoded_df.drop('TARGET', axis = 1)

# Feature names
#features = list(X.columns)

# Split into Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# KNN model
neigh = KNeighborsClassifier(n_neighbors=5)

# Train the model on the Training data
neigh.fit(X_train, y_train)

# Score the model on the Testing data
score = neigh.score(X_test, y_test) 
print( "score : ", score )

score :  0.9194137855419925


In [15]:

# KNN model
neigh = KNeighborsClassifier(n_neighbors=5)

# Train the model on the Training data
neigh.fit(X_train, y_train)

# cross validation 
# Test model performance 
score = cross_val_score(neigh, X_train, y_train, cv=5).mean() 
print("score : ", score)

#  Predict on new data
y_pred = neigh.predict(X_test)

# Test accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


score :  0.9214021659894115
Accuracy: 0.9194137855419925


# Classifier Models 

In [17]:
scoring = ['roc_auc', 'accuracy']
models = []
results = []

# Classifiers
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
models.append(('DTC', DecisionTreeClassifier()))
#models.append(('RF', RandomForestClassifier()))
#models.append(('SVC', SVC()))

In [18]:
for name, model in models:
    for s in scoring:
        model.fit(X_train, y_train)
        res = cross_val_score(model, X_train, y_train, cv=5, scoring=s).mean()
        results.append(res)
        print("Model: ", name, " scoring:", s, " score", res)

/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
   

Model:  LR  scoring: roc_auc  score 0.6130893675025983


/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
   

Model:  LR  scoring: accuracy  score 0.9258818373955877
Model:  KNN  scoring: roc_auc  score 0.5261843510796194
Model:  KNN  scoring: accuracy  score 0.9214021659894115
Model:  DTC  scoring: roc_auc  score 0.5319870075877831
Model:  DTC  scoring: accuracy  score 0.8626965840476357


# Prediction

 **Making prediction based on LR model that gives the best score**

In [20]:
model = LogisticRegression()
scoring = 'accuracy'

model.fit(X_train, y_train)

/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
#  Predict on new data
y_pred = model.predict(X_test)

# Test accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))   

Accuracy: 0.9239090709743439


# Modeling with scaled data

In [30]:
# Scaling data

# create X, y
y = encoded_df.TARGET
X = encoded_df.drop('TARGET', axis = 1)

# Scaling features
scaler = MinMaxScaler() # Instanciate StandarScaler
scaler.fit(X)

X_rescaled = scaler.transform(X)

# Split into Train/Test
X_train_sc, X_test_sc, y_train, y_test = train_test_split(X_rescaled, y, test_size=0.3)


In [31]:
scoring = ['roc_auc', 'accuracy']
models = []
results = []

# Classifiers
models.append(('LR', LogisticRegression(max_iter=1000)))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
models.append(('DTC', DecisionTreeClassifier()))

for name, model in models:
    for s in scoring:
        model.fit(X_train_sc, y_train)
        res = cross_val_score(model, X_train_sc, y_train, cv=5, scoring=s).mean()
        results.append(res)
        print("Model: ", name, " scoring:", s, " score", res)      

Model:  LR  scoring: roc_auc  score 0.7400404423400226
Model:  LR  scoring: accuracy  score 0.9253962741845608
Model:  KNN  scoring: roc_auc  score 0.5481678467183916
Model:  KNN  scoring: accuracy  score 0.9220443474965118
Model:  DTC  scoring: roc_auc  score 0.5361585969728132
Model:  DTC  scoring: accuracy  score 0.8607700002708627


**LogisticRegression model gives the best score**

**Let's make predictions on test data**

In [32]:
model = LogisticRegression(max_iter=1000)
scoring = 'accuracy'

model.fit(X_train_sc, y_train)
res = cross_val_score(model, X_train_sc, y_train, cv=5, scoring=s).mean()
print("score: ", res)  

#  Predict on new data
y_pred = model.predict(X_test_sc)

# Test accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))   

score:  0.9253962741845608
Accuracy: 0.9250420290914407


# Encoder() class

In [3]:
class Encoder():
    
    """ Initialize dataframe
    """
    def __init__(self):

        path_dir = (os.path.dirname(os.getcwd()))
        sys.path.append(path_dir)
        # Assign an attribute ".data" to all new instances of Preparation
        self.data = Cleaning().remove_missvalues() # good practice to be sure not to modify your `data` variable
        self.catcols = Preparation().get_catcols()
        self.numcols = Preparation().get_numcols() 
        
        
    def execute(self):
        
        copy = self.data.copy()
        
        for col_name in self.catcols:
    
            L = list(copy[col_name].unique())
            if '' in L:
                copy[col_name].replace("", "NoValue", inplace=True) #Replace NaN by "NoCodeNature"

            ohe = OneHotEncoder(sparse = False) # Instanciate encoder
            ohe.fit(copy[[col_name]]) # Fit encoder  ---> OneHotEncoder(sparse=False)

            col_encoded = ohe.transform(copy[[col_name]]) # Encode

            dicts_col = {}
            keys = list(ohe.categories_[0])
            values = col_encoded.T.astype(int)

            for i,j in enumerate(keys):
                dicts_col[j] = values[i,:]

            result = pd.DataFrame.from_dict(dicts_col)

            copy = copy.reset_index(drop=True)

            #Concat self.data and result dataframes
            data_res = pd.concat([copy, result], axis = 1)

            if 'NoValue' in list(data_res.columns):
                data_res = data_res.drop(columns= ['NoValue',col_name] )
                copy = data_res
            else:
                data_res = data_res.drop(columns= col_name)
                copy = data_res
        
        encoded_df = copy

        return encoded_df
        
 

In [4]:
en = Encoder()

/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [5]:
en.data.head(2)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
6,100009,0,Cash loans,F,Y,Y,1,171000.0,1560726.0,41301.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0


In [6]:
encoded_df = en.execute()
encoded_df.head(2)

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,Mixed,Monolithic,Others,Panel,"Stone, brick",Wooden,NaN,No,Yes,NaN
0,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,0,1,0,0,1
1,100009,0,1,171000.0,1560726.0,41301.0,1395000.0,0.035792,-13778,-3130,...,0,0,0,0,0,0,1,0,0,1


# Modeling() class

In [25]:

class Modeling:
    
    def __init__(self):
            
        path_dir = (os.path.dirname(os.getcwd()))
        sys.path.append(path_dir)
        # Assign an attribute ".data" to all new instances of Preparation
        self.data = Encoder().execute() # outliers and missing values are removed, encoding are done
        self.catcols = Preparation().get_catcols()
        self.numcols = Preparation().get_numcols()
        
    def preprocess(self): # we can here integrate scaler as arg
        
        #encoded_df = Encoder().execute()
        # create X, y
        y = self.data.TARGET
        X = self.data.drop('TARGET', axis = 1)

        # Scaling features
        scaler = MinMaxScaler() # Instanciate StandarScaler
        scaler.fit(X)

        X_rescaled = scaler.transform(X)

        # Split into Train/Test
        X_train_sc, X_test_sc, y_train, y_test = train_test_split(X_rescaled, y, test_size=0.3)
        
        return X_train_sc, X_test_sc, y_train, y_test
       
        
    def execute(self): # we can here integrate models, and scoring as arg
        
        X_train_sc, X_test_sc, y_train, y_test = self.preprocess()
        
        scoring = ['roc_auc', 'accuracy']
        
        models = []
        results = []

        # Classifiers
        models.append(('LR', LogisticRegression(max_iter=1000)))
        models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
        models.append(('DTC', DecisionTreeClassifier()))
        #models.append(('RF', RandomForestClassifier()))
        #models.append(('SVC', SVC()))

        for name, model in models:
            for s in scoring:
                model.fit(X_train_sc, y_train)
                res = cross_val_score(model, X_train_sc, y_train, cv=5, scoring=s).mean()
                results.append([name, s, res])
                print("Model: ", name, " scoring:", s, " score", res) 
                
        return {"Models": models, "Scoring":scoring, "Results": results}
    
    def predict(self, best_model, best_scoring :str):
        
        X_train_sc, X_test_sc, y_train, y_test = self.preprocess()
        
        best_model.fit(X_train_sc, y_train)
        res = cross_val_score(best_model, X_train_sc, y_train, cv=5, scoring=best_scoring).mean()
        print("score: ", res)  

        #  Predict on new data
        y_pred = best_model.predict(X_test_sc)

        # Test accuracy
        print("Accuracy:",metrics.accuracy_score(y_test, y_pred))  
        
        return metrics.accuracy_score(y_test, y_pred)



In [26]:
ml = Modeling()

/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [27]:
ml.data.head(2)

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,Mixed,Monolithic,Others,Panel,"Stone, brick",Wooden,NaN,No,Yes,NaN
0,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,0,1,0,0,1
1,100009,0,1,171000.0,1560726.0,41301.0,1395000.0,0.035792,-13778,-3130,...,0,0,0,0,0,0,1,0,0,1


In [28]:
ml.data.shape

(91206, 247)

In [29]:
dic = ml.execute()

Model:  LR  scoring: roc_auc  score 0.7457478275223101
Model:  LR  scoring: accuracy  score 0.9246601028081207
Model:  KNN  scoring: roc_auc  score 0.5421502379273351
Model:  KNN  scoring: accuracy  score 0.9206659897060374
Model:  DTC  scoring: roc_auc  score 0.5312529888622792
Model:  DTC  scoring: accuracy  score 0.8600651204132188


In [30]:
dic['Results']

[['LR', 'roc_auc', 0.7457478275223101],
 ['LR', 'accuracy', 0.9246601028081207],
 ['KNN', 'roc_auc', 0.5421502379273351],
 ['KNN', 'accuracy', 0.9206659897060374],
 ['DTC', 'roc_auc', 0.5312529888622792],
 ['DTC', 'accuracy', 0.8600651204132188]]

In [31]:
ml.predict(best_model = LogisticRegression(max_iter=1000), best_scoring='accuracy')

score:  0.9243625082951722
Accuracy: 0.9273079453256341


0.9273079453256341